# Multitask Learning for BERT Example
In this example RuBERT Base is trained on DaNetQA task of Russian SuperGLUE and sentiment analysis task on "mokoron" dataset of tweets

In [1]:
from task import Task
import transformers
from rutasks import rsg, mokoron
import datasets
from data_utils import dict_map
from multitask_transformers import MultitaskModel, make_features, MultitaskTrainer, NLPDataCollator, unpack_splits

Tasks are created using Task(...) with `cls` - model class from `transformers` (Huggingface Transformers module), `config` from `transformers`, `data` dataset with parts from `datasets`(Huggingface Datasets module), `converter_to_features` being a function mapping one batch to defaultish features usually fed into models of `transformers` and an optional `name` for verbose messages

In [2]:
base_model_name = "DeepPavlov/rubert-base-cased"
tokenizer = transformers.AutoTokenizer.from_pretrained(base_model_name)
conv_params = {"padding": True, "truncation": True, "max_length": 512}
tasks = {
    'danetqa': Task(
        cls=transformers.AutoModelForSequenceClassification,
        config=transformers.AutoConfig.from_pretrained(base_model_name, num_labels = 2),
        data=dict_map(datasets.load_dataset("russian_super_glue", 'danetqa'), rsg.preprocess_danetqa, name="danetqa"),
        converter_to_features=rsg.InputLabelConv(tokenizer, **conv_params),
        name=f"Russian SuperGLUE: DaNetQA"
    ),
    'mokoron': Task(
        cls=transformers.AutoModelForSequenceClassification,
        config=transformers.AutoConfig.from_pretrained(base_model_name, num_labels = 2),
        data=mokoron.load(),
        converter_to_features=rsg.InputLabelConv(tokenizer, **conv_params)
    )
}

Reusing dataset russian_super_glue (/Users/s1m00n/.cache/huggingface/datasets/russian_super_glue/danetqa/0.0.1/6fcadbfc1d8f0298b2f01ff277093772efe9e1b98f3c0df8ab5f511b3b9e13c9)


  0%|          | 0/3 [00:00<?, ?it/s]

map(preprocess_danetqa, danetqa):
    map(preprocess_danetqa, danetqa[train])
    map(preprocess_danetqa, danetqa[validation])
    map(preprocess_danetqa, danetqa[test])


Making multitask features and unpack splits for each task

In [ ]:
features = make_features(tasks)
train, validation = unpack_splits(features, "train", "validation")

In [ ]:
# temp
train, validation = unpack_splits(features, "train", "validation")

In [ ]:
model = MultitaskModel.create(base_model_name, tasks)
trainer = MultitaskTrainer(
    model=model,
    args=transformers.TrainingArguments(
        output_dir="test",
        overwrite_output_dir=True,
        learning_rate=1e-5,
        logging_steps=100,
        eval_steps=1000,
        do_train=True,
        num_train_epochs=1,
        per_device_train_batch_size=15,
        per_device_eval_batch_size=128, # TODO: this value is not read in MultitaskTrainer (strange bug)
        save_steps=5000,
    ),
    data_collator=NLPDataCollator(),
    train_dataset=train
    eval_dataset=validation
)
trainer.train()